In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder,MinMaxScaler

In [ ]:
df_daily_calendar= pd.read_csv("daily_calendar_with_events.csv")
df_item_prices=pd.read_csv("item_prices.csv")
df_item_sales=pd.read_csv("item_sales.csv")
df_daily_calendar.info()